In [1]:
# the following lines disable the numpy multithreading [optional]
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import KitBase as kt

import matplotlib.pyplot as plt

from examples.nonlinear.boltzmann_3d_pbc_upwind1 import Boltzmann
prob = Boltzmann()

┌ Warning: Replacing docs for `SciMLBase.sol :: Union{Tuple, Tuple{D}, Tuple{S}, Tuple{N}, Tuple{T}} where {T, N, S, D}` in module `SciMLBase`
└ @ Base.Docs docs/Docs.jl:240
[ Info: Kinetic will run with 1 worker and 1 thread
usage: ipykernel_launcher.py [-h] [--T_start T_START] [--T_end T_END]
                             [--beta BETA] [--rolling ROLLING]
                             [--time_intervals TIME_INTERVALS]
                             [--time_points TIME_POINTS] [--proc_row PROC_ROW]
                             [--proc_col PROC_COL]
                             [--spatial_points SPATIAL_POINTS [SPATIAL_POINTS ...]]
                             [--solver SOLVER] [--maxiter MAXITER] [--tol TOL]
                             [--stol STOL] [--smaxiter SMAXITER]
                             [--document DOCUMENT]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/gaya/.local/share/jupyter/runtime/kernel-73df054c-0614-40f8-8a51-ce9777803942.json


SystemExit: 2

/home/gaya/miniconda3/envs/env3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# for Jupyter
prob.document = 'None'
prob.betas = [0]

In [ ]:
# choosing a number of points
prob.spatial_points = [100, 48, 28, 28]
prob.time_points = 3

# choosing a time domain
prob.T_start = 0

# choosing the number of intervals handled in parallel
prob.time_intervals = 1
prob.rolling = 8

prob.T_end = 8e-2 #prob.rolling * prob.time_intervals

# choosing a parallelization strategy
prob.proc_col = 1
prob.proc_row = prob.time_intervals

# choosing a solver
prob.solver = 'gmres'

# setting maximum number of iterations
prob.maxiter = 10
prob.smaxiter = 50

# choosing a setting for the alpha sequence
prob.alphas = [1e-6]

# setting tolerances
prob.tol = 1e-4
prob.stol = 1e-6

prob.setup()
prob.solve()
prob.summary(details=True)

In [ ]:
import numpy as np
prob.__fill_u_last__(fill_old=False)

w0 = np.zeros((prob.spatial_points[0], 5))
w = np.zeros_like(w0)

f0 = prob.u_initial(0)
f = prob.u_last_loc.reshape(prob.spatial_points)

for i in range(prob.spatial_points[0]):
    w0[i, :] = kt.moments_conserve(f0[i, :, :, :].real, prob.vs.u, prob.vs.v, prob.vs.w, prob.vs.weights)
    w[i, :] = kt.moments_conserve(f[i, :, :, :].real, prob.vs.u, prob.vs.v, prob.vs.w, prob.vs.weights)
    
plt.plot(w[:, 1])
plt.plot(w0[:, 1], '--')
plt.legend(['w', 'w0'])
plt.savefig('w_and_w0.png', dpi=500)
plt.show()
    